In [53]:
### Necessary Libraries
# from msilib.schema import File
import pandas as pd, unicodedata
import string as strlab
import random, itertools
import os    
from csv import DictWriter
from datetime import date
import psycopg2
import numpy as np
from sqlalchemy import create_engine
import re 
import sys
%config IPCompleter.use_jedi=False
import datetime

In [54]:
def get_db_creds(dbname='pricedbprod',user='nextorbitprod',password='5ASyeJGXpjFFRA8k6xkj'):
    # Connect to the database
    host = "pricedbprod.c6efngmeow34.ap-south-1.rds.amazonaws.com"
    port = "5432"
    engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{dbname}")
    conn = psycopg2.connect(database=dbname, user=user, password=password, host=host, port=port)
    return conn

def get_tabledata(schema,tablename,conn):
    query = f'SELECT * FROM {schema}."{tablename}"'
    table = pd.read_sql_query(query,conn)
    return table


In [55]:
conn = get_db_creds()
product_master = get_tabledata(schema='consolidated',tablename='product_master_20220406',conn=conn)


In [56]:
updates_data = pd.read_excel("C:\\Users\\Bhaskar\\OneDrive\\Desktop\\220411 Tag Revised v2.xlsx",sheet_name='Sheet1')

In [57]:
product_master=pd.merge(product_master,updates_data,left_on=['product_name_cleaned','brand_name'],right_on=['Product Name Cleaned','Brand Name'],how='left')
product_master.drop(columns=['system_display','Product Name Cleaned','major_category','minor_category','Brand Name','display_name'],inplace=True)
product_master.rename(columns={'Minor Category v2':'minor_category','Major Category v2':'major_category','Meal Period':'meal_period','Display Name':'display_name','System Display':'system_display'},inplace=True)


In [58]:
dbname='pricedbprod'
user='nextorbitprod'
password='5ASyeJGXpjFFRA8k6xkj'
port="5432"
host = "pricedbprod.c6efngmeow34.ap-south-1.rds.amazonaws.com"

engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{dbname}")



In [60]:
product_master.to_sql('product_master_20220422',con=engine,schema='consolidated',if_exists='append',index=False) 